# Project 1: Poker Data
### September 27, 2015

Insert narrative and some instructions here.

In [198]:
# Parses "hdb" file from the IRC Poker Database http://poker.cs.ualberta.ca/irc_poker_database.html
import xlrd
import os
import csv
from zipfile import ZipFile
import pprint

hdb_file = "./nolimit/199505/hdb"
hroster_file = "./nolimit/199505/hroster"
#pdb_file = "./nolimit/199505/pdb/pdb.ShoelessJ"
#pdf_directory = "./nolimit/199505/pdb/"
outfile = "./nolimit/199505/hands_output.txt"

#def open_zip(datafile):
#    with ZipFile('{0}.zip'.format(datafile), 'r') as myzip:
#        myzip.extractall()

# Global variables
pot_cats = ["flop", "turn", "river", "showdown"]
deck = {'A': 'ace', 'K': 'king', 'Q': 'queen', 'J': 'jack', 'T': '10'}
suits = {'c': 'clubs', 's': 'spades', 'h': 'hearts', 'd': 'diamonds'}
bet_action_codes = {
            '-' : 'no action',
            'B' : 'blind bet',
            'f' : 'fold',
            'k' : 'check',
            'b' : 'bet',
            'c' : 'call',
            'r' : 'raise',
            'A' : 'all-in',
            'Q' : 'quits game',
            'K' : 'kicked from game'
            }
bet_action_cats = ["pre-flop", "flop", "turn", "river"]


def parse_hdb_file(hdb_file):
    
    hands = {}
    with open(hdb_file, "r") as hdb:
        for line in hdb:
            hand = {}
            pot_data = []
            board_card_data = []
            timestamp = line[:9]
            hand["dealer"] = line[10:13].strip(" ")
            hand["hand_num"] = line[14:19].strip(" ")
            hand["num_players"] = line[20:22].strip(" ")
            pot_data.append(line[23:31].strip(" "))
            pot_data.append(line[31:39].strip(" "))
            pot_data.append(line[39:48].strip(" "))
            pot_data.append(line[48:57].strip(" "))
            board_card_data.append(line[58:60].strip(" ").strip("\n"))
            board_card_data.append(line[61:63].strip(" ").strip("\n"))
            board_card_data.append(line[64:66].strip(" ").strip("\n"))
            board_card_data.append(line[67:69].strip(" ").strip("\n"))
            board_card_data.append(line[70:72].strip(" ").strip("\n"))
            
            pots = []
            i = 0
            for p in pot_data:
                pot = {}
                pot["stage"] = pot_cats[i]
                pot["num_players"] = p.split("/")[0]
                pot["starting_pot_size"] = p.split("/")[1]
                pots.append(pot)
                i = i + 1
                
            board = []
            for b in board_card_data:
                board_card = {}
                if b != "":
                    if board_card_data.index(b) + 1 <= 3:
                        board_card["stage"] = "flop"
                    elif board_card_data.index(b) + 1 == 4:
                        board_card["stage"] = "turn"
                    elif board_card_data.index(b) + 1 == 5:
                        board_card["stage"] = "river"
                    if b[0] in deck.keys():
                        board_card["value"] = deck[b[0]]
                    else:
                        board_card["value"] = b[0]
                    board_card["suit"] = suits[b[1]]
                    board.append(board_card)
                
            hand["pots"] = pots
            hand["board"] = board
            hands[timestamp] = hand
    hdb.close()
    return hands


def parse_hroster_file(hroster_file, hands):
    
    with open(hroster_file, "r") as hroster:
        for line in hroster:
            players = {}
            timestamp = line[:9]
            player_data = line[13:].strip("\n").split(" ")
            for p in player_data:
                player = {}
                player["username"] = p
                players[p] = player
            hands[timestamp]["players"] = players
    hroster.close()
    return hands

    
def parse_pdb_file(pdb_file, hands):
    
    with open(pdb_file, "r") as pdb:
        filename = pdb_file.split("/")
        username = filename[len(filename)-1].split(".")[1]
        for line in pdb:
            timestamp = line[10:19]
            position = line[23:25].strip(" ")
            bet_action_rounds = line[26:47].split(" ")
            bet_action_rounds = [elem for elem in bet_action_rounds if elem != '']
            bet_actions = []
            i = 0
            for bet_action_round in bet_action_rounds:
                bet_action = {}
                bet_action["actions"] = []
                for b in bet_action_round:
                    bet_action["actions"].append(bet_action_codes[b])
                bet_action["stage"] = bet_action_cats[i]
                bet_actions.append(bet_action)
                i = i + 1
                
            other_info = line[48:].strip("\n").split(" ")
            other_info = [elem for elem in other_info if elem != '']
            i = 0
            player_cards = []
            for item in other_info:
                player_card = {}
                if i == 0: bankroll = other_info[i]
                if i == 1: action = other_info[i]
                if i == 2: winnings = other_info[i]
                if i >= 3:
                    if item[0] in deck.keys():
                        player_card["value"] = deck[item[0]]
                    else:
                        player_card["value"] = item[0]
                    player_card["suit"] = suits[item[1]]
                    player_cards.append(player_card)
                i = i + 1
            hands[timestamp]["players"][username]["bet_actions"] = bet_actions
            hands[timestamp]["players"][username]["bankroll"] = int(bankroll)
            hands[timestamp]["players"][username]["action"] = int(action)
            hands[timestamp]["players"][username]["winnings"] = int(winnings)
            hands[timestamp]["players"][username]["player_cards"] = player_cards
            hands[timestamp]["players"][username]["position"] = int(position)
#            pprint.pprint(hands[timestamp]["players"])
    pdb.close()
    return hands

    
def loop_pdb_files(hands_col):
    for root, dirs, files in os.walk(".", topdown=False):
        for name in files:
            full_path_directories = root.split("/")
            if len(full_path_directories) >= 4:
                if full_path_directories[1] == "nolimit" and full_path_directories[3] == "pdb":
                    pdb_file = os.path.join(root, name)
                    print "Processing " + pdb_file + "..."
                    hs = parse_pdb_file(pdb_file, hands_col)
    print "...Finished processing PDB files."
    return hands_col
    

hs = parse_hdb_file(hdb_file)
hs = parse_hroster_file(hroster_file, hs)
hs = loop_pdb_files(hs)
pprint.pprint(hs['800199311'])

# Still to do 9/28/15:
# Download the files directly from the Internet (http://poker.cs.ualberta.ca/IRCdata/)
# Loop through hdb and hroster files, be sure to tack YearMonth onto front of timestamp field
# Loop through tgz files
# Change working directory?

Processing ./nolimit/199505/pdb/pdb.deadhead...
Processing ./nolimit/199505/pdb/pdb.greg...
Processing ./nolimit/199505/pdb/pdb.jzkbot...
Processing ./nolimit/199505/pdb/pdb.ShoelessJ...
Processing ./nolimit/199505/pdb/pdb.tvp...
...Finished processing PDB files.
{'board': [{'stage': 'flop', 'suit': 'hearts', 'value': 'ace'},
           {'stage': 'flop', 'suit': 'hearts', 'value': '4'},
           {'stage': 'flop', 'suit': 'diamonds', 'value': '8'},
           {'stage': 'turn', 'suit': 'hearts', 'value': '5'},
           {'stage': 'river', 'suit': 'hearts', 'value': '8'}],
 'dealer': '1',
 'hand_num': '455',
 'num_players': '2',
 'players': {'ShoelessJ': {'action': 1400,
                           'bankroll': 8266,
                           'bet_actions': [{'actions': ['blind bet',
                                                        'raise'],
                                            'stage': 'pre-flop'},
                                           {'actions': ['check'],
        